# Why Julia?

# Quasi-Newton Method (BFGS)
Learning matrix operations in Julia through an example about Newton's method.<br>
We want to find out the stationary point of the following multi-variate function:
$$f(x)=(x_1-2)^4+(x_1-2x_2)^2$$


In [1]:
f(x1,x2)=(x1-2)^4+(x1-2x2)^2 #define functions (one of three ways)
df(x1,x2)=[4(x1-2)^3+2(x1-2x2);-4(x1-2x2)]
Hessian(x1,x2)=[12(x1-2)^2+2x1 -4;-4 8]

Hessian (generic function with 1 method)

In [2]:
inv(Hessian(2,3))#getting the inverse of a matrix. Don't need additional package

2×2 Array{Float64,2}:
 0.5   0.25
 0.25  0.25

In [5]:
#initializing variables
x=[0;0]
f_now=f(x[1],x[2]) #in julia, indices start from 1, instead of 0
f_prev=-Inf
df_now=df(x[1],x[2])
H=Hessian(x[1],x[2])
ϵ=1e-08 #cool feature: \epsilon then Tab to get the greek letter
α=1 
Nloop=100
k=0

while k<=Nloop #a while loop
    if abs(f_now-f_prev)>ϵ #using if
        k+=1
        f_prev=f_now
        df_prev=df_now
        d=-inv(H)*df_now
        x=x+α*d
        f_now=f(x[1],x[2])
        df_now=df(x[1],x[2])
        s=α*d
        y=df_now-df_prev
        H=H+y*y'/(y'*s)-(H*s*s'*H)/(s'*H*s) #use ' to get transpose of a matrix
    else 
        break
    end
end
if abs(f_now-f_prev)>ϵ
    println("BFGS fails to converge winthin 100 steps") #print text and change line
else
    @printf("BFGS converges within %d steps",k) #print formatted text. Doesn't change line automatically
    println("The optimal value is ", f_now)
end   

BFGS converges within 20 stepsThe optimal value is 1.7570295550248682e-9


In [8]:
#put everything inside function so we can change starting point easily
function f(m_x,m_loop,m_α)
        #initializing variables
    x=m_x
    f_now=f(x[1],x[2]) #in julia, indices start from 1, instead of 0
    f_prev=-Inf
    df_now=df(x[1],x[2])
    H=Hessian(x[1],x[2])
    ϵ=1e-08 #cool feature: \epsilon then Tab to get the greek letter
    α=m_α 
    Nloop=m_loop
    k=0

    while k<=Nloop #a while loop
        if abs(f_now-f_prev)>ϵ #using if
            k+=1
            f_prev=f_now
            df_prev=df_now
            d=-inv(H)*df_now
            x=x+α*d
            f_now=f(x[1],x[2])
            df_now=df(x[1],x[2])
            s=α*d
            y=df_now-df_prev
            H=H+y*y'/(y'*s)-(H*s*s'*H)/(s'*H*s) #use ' to get transpose of a matrix
        else 
            break
        end
    end
    if abs(f_now-f_prev)>ϵ
        println("BFGS fails to converge winthin 100 steps") #print text and change line
    else
        @printf("BFGS converges within %d steps",k) #print formatted text. Doesn't change line automatically
        println("The optimal value is ", f_now)
    end   
end

f (generic function with 3 methods)

In [9]:
f([1;1],100,1)

BFGS converges within 17 stepsThe optimal value is 3.815550033188953e-9


In [12]:
f([1;1],100,0.1) #fails to converge. Change m_loop to 1000

BFGS fails to converge winthin 100 steps


# A Simple Optimization Problem
JuMP is very useful when solving simple optimization for homework/fun. Or when you want to check the feasibility of a linear system.
$$\begin{aligned}
\min\quad &x_1+x_2\\
s.t.\quad &x_1+3x_2\geq 1\\
 &x_1,x_2>=0
\end{aligned}
$$

In [13]:
#using packages
#JuMP is the package for optimization. Gurobi is the solver wrapper for the solver Gurobi
using JuMP, Gurobi 

In [15]:
#defining a model object, m
m=Model(solver=GurobiSolver())

#defining variables
@variables m begin
    x1>=0
    x2>=0
end

#defining objective
@objective(m,Min,x1+x2)

#defining constraints
@constraint(m,x1+3x2>=1)

solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x2= ", getvalue(x2))

Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3333333e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  3.333333333e-01
Objective value: 0.3333333333333333
x2= 0.3333333333333333


# Solving a Facility Location Problem Using JuMP


Reference:<br>
[1] 